In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re
import nltk
from nltk.corpus import stopwords
from src.config import CUSTOM_STOPWORDS, CATEGORIES
from src.utils.helpers import clean_text
from src.utils.helpers import assign_label

In [28]:
import pandas as pd
import nltk
from src.utils.helpers import clean_text, assign_label

# Télécharger les stopwords (une seule fois)
nltk.download('stopwords')

# === Étape 1 : Chargement des données brutes ===
raw_path = "data/raw/raw_data.csv"
processed_path = "data/processed/processed_data.csv"

print("Chargement des données...")
df = pd.read_csv(raw_path, sep=',', on_bad_lines='skip')

# === Étape 2 : Nettoyage des colonnes ===
df.drop(columns=["location", "Id_reviews", "published_date"], inplace=True, errors='ignore')

# Fusion des colonnes 'reviews' et 'title_reviews'
df['reviews'] = df['reviews'].fillna(df['title_reviews'])
df.dropna(subset=['reviews'], inplace=True)

# === Étape 3 : Nettoyage du texte ===
print("Nettoyage du texte...")
df['clean_reviews'] = df['reviews'].astype(str).apply(clean_text)
df = df.drop(df[df["clean_reviews"].isna()].index)

# === Étape 4 : Attribution de labels thématiques ===
print("Attribution des catégories...")
df['category'] = df['clean_reviews'].apply(assign_label)

# === Étape 5 : Sauvegarde du fichier nettoyé ===
print(f"Enregistrement des données prétraitées dans {processed_path}")
df.to_csv(processed_path, index=False)
print("Prétraitement terminé.")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ramil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Chargement des données...
Nettoyage du texte...
Attribution des catégories...
Enregistrement des données prétraitées dans data/processed/processed_data.csv
Prétraitement terminé.


In [29]:
data = pd.read_csv(processed_path)
data

,company,trust_score,reviews_number,keywords,title_reviews,reviews,score_reviews,clean_reviews,category
0,Canebiera,4.9,249.0,"Production agricole, ·, Magasin de vitamines e...",QUALITE!!!,Très haute qualité!Bonne sélection de variétés...,5.0,haute qualité sélection variétés différentes u...,Recommandation
1,Canebiera,4.9,249.0,"Production agricole, ·, Magasin de vitamines e...",beaucoup,"beaucoup, de progrès pour ces produits ,tres...",5.0,beaucoup progrès produits produit,Qualité/Prix
2,Canebiera,4.9,249.0,"Production agricole, ·, Magasin de vitamines e...",Bonne réactivité,UPS a repoussé ma livraison 3 fois et à finale...,5.0,ups a repoussé livraison fois finalement perdu...,Livraison
3,Canebiera,4.9,249.0,"Production agricole, ·, Magasin de vitamines e...",Toujours aussi satisfait de la qualité…,Toujours aussi satisfait de la qualité des her...,5.0,toujours aussi satisfait qualité herbes créé c...,Qualité/Prix
4,Canebiera,4.9,249.0,"Production agricole, ·, Magasin de vitamines e...",Toujours aussi fan..,"Toujours aussi fan... Selon moi, c'est LA sour...",5.0,toujours aussi fan selon source marché tout re...,Recommandation
...,...,...,...,...,...,...,...,...,...
118873,Wikicampers,4.4,390.0,"Vendeur de voitures d'occasion, ·, Location de...",Superbe première expérience en van,"Superbe première expérience en van, la réserva...",5.0,superbe première expérience van réservation a ...,Satisfaction Générale
118874,Wikicampers,4.4,390.0,"Vendeur de voitures d'occasion, ·, Location de...",Super expérience.,Super expérience.Très bon contact avec Jean-Sé...,5.0,super expérience contact jean sébastien véhicu...,Satisfaction Générale
118875,Wikicampers,4.4,390.0,"Vendeur de voitures d'occasion, ·, Location de...",Manque de vigilance de l'expert de Wikicamper,Après quelques jours d'utilisation du camping ...,2.0,après quelques jours utilisation camping car a...,Autre
118876,Wikicampers,4.4,390.0,"Vendeur de voitures d'occasion, ·, Location de...","Cosy, propre, fonctionnel, facile à conduire.",C'était notre première expérience en fourgon :...,5.0,première expérience fourgon accueil explicatio...,Satisfaction Générale


In [31]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Séparer les features (X) et les labels (y)
X = data["clean_reviews"]
y = data["category"]

# Diviser en jeu d'entraînement et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Remplacer les NaN par des chaînes vides
X_train = X_train.fillna("")
X_test = X_test.fillna("")

In [ ]:
# Transformer les avis en vecteurs numériques
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [34]:
# Classification RandomForest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_tfidf, y_train)
y_pred = model.predict(X_test_tfidf)

# Évaluation du modèle
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9817883580080754
                       precision    recall  f1-score   support

                Autre       0.98      1.00      0.99      8498
 Formation/Entreprise       0.97      0.98      0.97      1008
            Livraison       1.00      0.97      0.98      5128
         Qualité/Prix       0.98      0.98      0.98      3652
       Recommandation       0.98      0.98      0.98      1157
Satisfaction Générale       0.97      0.95      0.96      1770
       Service Client       0.99      0.98      0.99      2563

             accuracy                           0.98     23776
            macro avg       0.98      0.98      0.98     23776
         weighted avg       0.98      0.98      0.98     23776



In [39]:
import joblib
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
import pandas as pd


# Préparer les features et la cible
X = data['clean_reviews']
y = data['score_reviews'] # Cible : score_reviews pour la régression


# Diviser les données en training et test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Remplacer les NaN par des chaînes vides
X_train = X_train.fillna("")
X_test = X_test.fillna("")

# Vectorisation du texte 
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


# Initialiser et entraîner un modèle de régression linéaire
regressor = LinearRegression()
regressor.fit(X_train_tfidf, y_train)

# Prédictions
y_pred = regressor.predict(X_test_tfidf)

# Évaluation du modèle
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

# Afficher les performances
print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")

# Sauvegarder le modèle
#joblib.dump(regressor, 'regression_model.joblib')

# Afficher quelques prédictions pour vérifier
predictions = pd.DataFrame({'True': y_test, 'Predicted': y_pred})
print(predictions.head())


Mean Absolute Error: 0.45
Mean Squared Error: 0.47
        True  Predicted
83678    5.0   4.270154
95218    5.0   4.855024
103981   5.0   4.999413
47537    3.0   2.449094
64302    5.0   4.675022


In [40]:
import pandas as pd
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression

# 1. Charger les données
data_path = "data/processed/processed_data.csv"
data = pd.read_csv(data_path)

# Features
X_text = data["clean_reviews"].fillna("")

# === Classification ===
print("== Entraînement modèle de classification (catégories) ==")
y_class = data["category"]
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_text, y_class, test_size=0.2, random_state=42)

vectorizer_class = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
X_train_vec_class = vectorizer_class.fit_transform(X_train_class)
X_test_vec_class = vectorizer_class.transform(X_test_class)

classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train_vec_class, y_train_class)

# Sauvegarde
joblib.dump(classifier, "models/random_forest/model.pkl")
joblib.dump(vectorizer_class, "models/random_forest/vectorizer.pkl")

print("Modèle de classification entraîné et sauvegardé.")


# === Régression ===
print("== Entraînement modèle de régression (score) ==")
y_reg = data["score_reviews"]
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_text, y_reg, test_size=0.2, random_state=42)

vectorizer_reg = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
X_train_vec_reg = vectorizer_reg.fit_transform(X_train_reg)
X_test_vec_reg = vectorizer_reg.transform(X_test_reg)

regressor = LinearRegression()
regressor.fit(X_train_vec_reg, y_train_reg)

# Sauvegarde
joblib.dump(regressor, "models/linear_regression/model.pkl")
joblib.dump(vectorizer_reg, "models/linear_regression/vectorizer.pkl")

print("Modèle de régression entraîné et sauvegardé.")


== Entraînement modèle de classification (catégories) ==
Modèle de classification entraîné et sauvegardé.
== Entraînement modèle de régression (score) ==
Modèle de régression entraîné et sauvegardé.


In [41]:
import pandas as pd
import joblib
import os
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    mean_absolute_error,
    mean_squared_error
)

# === Charger les données ===
data_path = "data/processed/processed_data.csv"
data = pd.read_csv(data_path)
X = data["clean_reviews"].fillna("")

# === CLASSIFICATION ===
print("== Évaluation du modèle de classification ==")
y_class = data["category"]
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(X, y_class, test_size=0.2, random_state=42)

# Charger modèle et vecteur
classifier = joblib.load("models/random_forest/model.pkl")
vectorizer_cls = joblib.load("models/random_forest/vectorizer.pkl")

X_test_vec_cls = vectorizer_cls.transform(X_test_cls)
y_pred_cls = classifier.predict(X_test_vec_cls)

# Évaluer
accuracy = accuracy_score(y_test_cls, y_pred_cls)
report = classification_report(y_test_cls, y_pred_cls, output_dict=True)

# Sauvegarder les métriques
metrics_class_path = "models/random_forest/metrics.json"
with open(metrics_class_path, "w") as f:
    json.dump({
        "accuracy": accuracy,
        "classification_report": report
    }, f, indent=4)

print(f"Classification accuracy: {accuracy:.2f}")
print(f"Rapport enregistré dans {metrics_class_path}")


# === RÉGRESSION ===
print("== Évaluation du modèle de régression ==")
y_reg = data["score_reviews"]
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y_reg, test_size=0.2, random_state=42)

# Charger modèle et vecteur
regressor = joblib.load("models/linear_regression/model.pkl")
vectorizer_reg = joblib.load("models/linear_regression/vectorizer.pkl")

X_test_vec_reg = vectorizer_reg.transform(X_test_reg)
y_pred_reg = regressor.predict(X_test_vec_reg)

# Évaluer
mae = mean_absolute_error(y_test_reg, y_pred_reg)
mse = mean_squared_error(y_test_reg, y_pred_reg)

# Sauvegarder les métriques
metrics_reg_path = "models/linear_regression/metrics.json"
with open(metrics_reg_path, "w") as f:
    json.dump({
        "mae": mae,
        "mse": mse
    }, f, indent=4)

print(f"Régression - MAE: {mae:.2f} | MSE: {mse:.2f}")
print(f"Rapport enregistré dans {metrics_reg_path}")


== Évaluation du modèle de classification ==
Classification accuracy: 0.98
Rapport enregistré dans models/random_forest/metrics.json
== Évaluation du modèle de régression ==
Régression - MAE: 0.45 | MSE: 0.47
Rapport enregistré dans models/linear_regression/metrics.json
